In [1]:
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
import pandas as pd
import openpyxl
import os

In [155]:
file_name = "/Users/luizacampos/Downloads/wbb_rosters_2022_23 (1).xlsx"
file_path = os.path.join(os.getcwd(), file_name)
df= pd.read_excel(file_path, sheet_name="wbb_rosters_2022_23")


Propuesta william

In [366]:
#El file name va a variar depende de la ubicación donde cada quien haya guardado el archivo
#pero la ruta que si debemos de tener todos en común es la siguiente:
#Predictive-Sentiment-Analysis-of-Twitter-for-BTC/Data_Analysis/Tableau_Practice/Data_Sets/wbb_rosters_2022_23.xlsx"
file_name = "C:/Users/willi/OneDrive/Escritorio/Henry/Proyecto twitter/Predictive-Sentiment-Analysis-of-Twitter-for-BTC/Data_Analysis/Tableau_Practice/Data_Sets/wbb_rosters_2022_23.xlsx"
file_path = os.path.join(os.getcwd(), file_name)
df= pd.read_excel(file_path, sheet_name="wbb_rosters_2022_23")

In [367]:
df.head()

,ncaa_id,team,player_id,name,year,hometown,homestate,high_school,previous_school_clean,height_clean,...,total_inches,primary_position,secondary_position,position_clean,year_clean,redshirt,hs_clean,hometown_clean,state_clean,country_clean
0,721,Air Force,11807.0,Mackenzie Le,Freshman,Elk Grove,California,St. Francis,NaN,"5'3""",...,63.0,GUARD,NaN,GUARD,Freshman,0,St. Francis,"ELK GROVE, CA",CA,USA
1,721,Air Force,11805.0,Milahnie Perry,Freshman,Tampa,Florida,Seffner Christian Academy,NaN,"5'7""",...,67.0,GUARD,NaN,GUARD,Freshman,0,Seffner Christian Academy,"TAMPA, FL",FL,USA
2,721,Air Force,11801.0,Madison Smith,Sophomore,Connell,Washington,Connell,NaN,"5'8""",...,68.0,GUARD,NaN,GUARD,Sophomore,0,Connell,"CONNELL, WA",WA,USA
3,721,Air Force,11795.0,Taylor Britt,Junior,Columbia,South Carolina,Spring Valley,USAFA Prep,"5'7""",...,67.0,GUARD,NaN,GUARD,Junior,0,Spring Valley,"COLUMBIA, SC",SC,USA
4,721,Air Force,11796.0,Kamri Heath,Senior,Edmond,Oklahoma,Edmond North,NaN,"5'7""",...,67.0,GUARD,NaN,GUARD,Senior,0,Edmond North,"EDMOND, OK",OK,USA


In [368]:
print(df.columns)

Index(['ncaa_id', 'team', 'player_id', 'name', 'year', 'hometown', 'homestate',
       'high_school', 'previous_school_clean', 'height_clean', 'position',
       'jersey', 'url', 'season', 'team_state', 'conference', 'division',
       'height_ft', 'height_in', 'total_inches', 'primary_position',
       'secondary_position', 'position_clean', 'year_clean', 'redshirt',
       'hs_clean', 'hometown_clean', 'state_clean', 'country_clean'],
      dtype='object')


Primero decidimos eliminar las columnas que no nos van a servir para el dashboard. Lo hicimos usando el metodo drop. Está por partes pues este método solo toma de 1 a 8 argumentos y queríamos eliminas muchas columnas

In [369]:
df = df.drop(['ncaa_id','player_id', 'year', 'hometown', 'homestate',
       'high_school', 'previous_school_clean'], axis=1)


In [370]:
df = df.drop([ 'url', 'season', 'team_state', 'conference', 'division',
       'height_ft', 'height_in', 'total_inches'], axis = 1)

In [371]:
df = df.drop(['year_clean', 'redshirt', 'jersey',
       'hs_clean', 'hometown_clean', 'state_clean'], axis=1)

In [372]:
df.head()

,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
0,Air Force,Mackenzie Le,"5'3""",G,GUARD,NaN,GUARD,USA
1,Air Force,Milahnie Perry,"5'7""",G,GUARD,NaN,GUARD,USA
2,Air Force,Madison Smith,"5'8""",G,GUARD,NaN,GUARD,USA
3,Air Force,Taylor Britt,"5'7""",G,GUARD,NaN,GUARD,USA
4,Air Force,Kamri Heath,"5'7""",G,GUARD,NaN,GUARD,USA


In [373]:
df.drop(['secondary_position'], axis=1)
df.head()

,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
0,Air Force,Mackenzie Le,"5'3""",G,GUARD,NaN,GUARD,USA
1,Air Force,Milahnie Perry,"5'7""",G,GUARD,NaN,GUARD,USA
2,Air Force,Madison Smith,"5'8""",G,GUARD,NaN,GUARD,USA
3,Air Force,Taylor Britt,"5'7""",G,GUARD,NaN,GUARD,USA
4,Air Force,Kamri Heath,"5'7""",G,GUARD,NaN,GUARD,USA


En un segundo paso, averiguamos valores nulos

In [374]:
print(df.isnull().sum())

team                      0
name                      0
height_clean             39
position                 25
primary_position         32
secondary_position    12787
position_clean           32
country_clean             0
dtype: int64


Eliminamos duplicados

In [375]:
df = df.drop_duplicates()

In [376]:
df.head()

,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
0,Air Force,Mackenzie Le,"5'3""",G,GUARD,NaN,GUARD,USA
1,Air Force,Milahnie Perry,"5'7""",G,GUARD,NaN,GUARD,USA
2,Air Force,Madison Smith,"5'8""",G,GUARD,NaN,GUARD,USA
3,Air Force,Taylor Britt,"5'7""",G,GUARD,NaN,GUARD,USA
4,Air Force,Kamri Heath,"5'7""",G,GUARD,NaN,GUARD,USA


Para el dashboard que vamos a armar y considerando la pequeña cantidad de valores nulos y su poca relevancia para el resultado final, decidimos eliminar los datos nulos y NaN en este archivo

In [377]:
data = df.dropna(how='any')


In [378]:
df.dropna(subset=['height_clean'], inplace=True)

In [379]:
df.head()

,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
0,Air Force,Mackenzie Le,"5'3""",G,GUARD,NaN,GUARD,USA
1,Air Force,Milahnie Perry,"5'7""",G,GUARD,NaN,GUARD,USA
2,Air Force,Madison Smith,"5'8""",G,GUARD,NaN,GUARD,USA
3,Air Force,Taylor Britt,"5'7""",G,GUARD,NaN,GUARD,USA
4,Air Force,Kamri Heath,"5'7""",G,GUARD,NaN,GUARD,USA


In [380]:
print(data.isna().sum())

team                  0
name                  0
height_clean          0
position              0
primary_position      0
secondary_position    0
position_clean        0
country_clean         0
dtype: int64


Finalmente, salvamos el archivo para usarlo en Tableau

In [256]:
df.to_excel(file_path, index=False)

Propuesta William

In [381]:
#propongo guardar el excel con otro nombre para mostrar que hay un avance en una limpieza de un excel a otro
df.to_excel('Data_Sets\wbb_rosters_2022_23_clean_data.xlsx', index=False)

Avance William

Obtener altura promedio por equipo


In [450]:
#observamos como los datos de la columna 'height_clean' estan en tipo str y con un tipo de apóstrofe 
#por lo que debemos de retirarlo y convertirlo a float para un análisis mas sencillo
df_altura_promedio = df.copy() #creamos otro data frame para no alterar el que ya fue trabajado
df_altura_promedio.head(3)

,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
0,Air Force,Mackenzie Le,"5'3""",G,GUARD,NaN,GUARD,USA
1,Air Force,Milahnie Perry,"5'7""",G,GUARD,NaN,GUARD,USA
2,Air Force,Madison Smith,"5'8""",G,GUARD,NaN,GUARD,USA


In [451]:
#verificamos dato por dato, uno a uno, si todos son del tipo str y encontramos que algunos son del tipo "datetime"
tipo_de_datos = df_altura_promedio['height_clean'].map(lambda x:type(x))
lista_tipo_de_datos = list(tipo_de_datos)
lista_tipo_de_datos [110:116] #para efectos practicos mandamos llamar los elementos del 110 al 116 para observar la diferencia

[str, str, str, datetime.datetime, datetime.datetime, datetime.datetime]

In [452]:
df_altura_promedio.loc[116,'height_clean'] #el dato en el en la celda index 116 de la columna 'height_clean' es una fecha

datetime.datetime(2023, 6, 5, 0, 0)

In [466]:
df_altura_promedio.iloc[110:116] #mejor visualización de los datos

,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
110,Boston College,T'Yana Todd,"6'0""",Guard,GUARD,NaN,GUARD,CANADA
111,Boston College,Ally Carman,"6'5""",Center,CENTER,NaN,CENTER,USA
112,Boston College,Akunna Konkwo,"6'2""",C,CENTER,NaN,CENTER,USA
121,Oregon State,Adlee Blacklock,6-0,Guard,GUARD,NaN,GUARD,USA
124,Belmont,Tuti Jones,"5'7""",Guard,GUARD,NaN,GUARD,USA
125,Belmont,Madison Bartley,"6'3""",Forward,FORWARD,NaN,FORWARD,USA


In [454]:
#cambiamos a nulo los datos que sean del tipo date time
df_altura_promedio['height_clean'] = df_altura_promedio['height_clean'].apply(lambda x: None if isinstance(x, pd.datetime) else x)
df_altura_promedio.head(3)

C:\Users\willi\AppData\Local\Temp\ipykernel_19688\255687533.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df_altura_promedio['height_clean'] = df_altura_promedio['height_clean'].apply(lambda x: None if isinstance(x, pd.datetime) else x)


,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
0,Air Force,Mackenzie Le,"5'3""",G,GUARD,NaN,GUARD,USA
1,Air Force,Milahnie Perry,"5'7""",G,GUARD,NaN,GUARD,USA
2,Air Force,Madison Smith,"5'8""",G,GUARD,NaN,GUARD,USA


In [465]:
df_altura_promedio.iloc[110:116] # aquí ya podemos observar como los datos que solían ser fechas pasaron a None

,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
110,Boston College,T'Yana Todd,"6'0""",Guard,GUARD,NaN,GUARD,CANADA
111,Boston College,Ally Carman,"6'5""",Center,CENTER,NaN,CENTER,USA
112,Boston College,Akunna Konkwo,"6'2""",C,CENTER,NaN,CENTER,USA
121,Oregon State,Adlee Blacklock,6-0,Guard,GUARD,NaN,GUARD,USA
124,Belmont,Tuti Jones,"5'7""",Guard,GUARD,NaN,GUARD,USA
125,Belmont,Madison Bartley,"6'3""",Forward,FORWARD,NaN,FORWARD,USA


In [456]:
#Volvemos a buscar los valores nulos del data frame pero lo pasamos a porcentaje
df_altura_promedio.isnull().mean()*100   # como solo es el 17.5% de los datos decidimos borrarlos

team                   0.000000
name                   0.000000
height_clean          17.586808
position               0.123493
primary_position       0.174343
secondary_position    92.612233
position_clean         0.174343
country_clean          0.000000
dtype: float64

In [457]:
df_altura_promedio = df_altura_promedio.dropna(subset=['height_clean'])  #eliminamos los valores null

In [458]:
df_altura_promedio.isnull().mean()*100 #observamos que ya no tenemos valores nulos para columna 'height_clean'

team                   0.000000
name                   0.000000
height_clean           0.000000
position               0.141031
primary_position       0.167475
secondary_position    93.027766
position_clean         0.167475
country_clean          0.000000
dtype: float64

In [459]:
#Revisamos de nuevo los datos uno a uno de la columna 'height_clean' y observamos 
# que hay datos que fueron mal cargados
datos_height_clean = df_altura_promedio['height_clean']
datos_height_clean = list(datos_height_clean)
datos_height_clean[110:119] #observamos de los datos 110 a 120 y el dato 114 es un ejemplo de mal cargado

['6\'0"',
 '6\'5"',
 '6\'2"',
 '6-0',
 '5\'7"',
 '6\'3"',
 '5\'8"',
 '5\'10"',
 '5\'8"']

In [464]:
#recordemos que el index con el que fue cargado es diferente al index de la posición en la tabla
# por lo que el index del dato 114 es el index de la posición 121 en la tabla
df_altura_promedio.iloc[110:119] #observamos como el dato esta mal cargado 6-0 en vez de 6'0

,team,name,height_clean,position,primary_position,secondary_position,position_clean,country_clean
110,Boston College,T'Yana Todd,"6'0""",Guard,GUARD,NaN,GUARD,CANADA
111,Boston College,Ally Carman,"6'5""",Center,CENTER,NaN,CENTER,USA
112,Boston College,Akunna Konkwo,"6'2""",C,CENTER,NaN,CENTER,USA
121,Oregon State,Adlee Blacklock,6-0,Guard,GUARD,NaN,GUARD,USA
124,Belmont,Tuti Jones,"5'7""",Guard,GUARD,NaN,GUARD,USA
125,Belmont,Madison Bartley,"6'3""",Forward,FORWARD,NaN,FORWARD,USA
126,Belmont,Brooke Highmark,"5'8""",Guard,GUARD,NaN,GUARD,USA
127,Belmont,Sydni Harvey,"5'10""",Guard,GUARD,NaN,GUARD,USA
128,Belmont,Blair Schoenwald,"5'8""",Guard,GUARD,NaN,GUARD,USA


In [449]:
#Revisamos de nuevo los datos uno a uno de la columna 'height_clean' y observamos 
# que hay datos que fueron mal cargados
datos_height_clean = df_altura_promedio['height_clean']
datos_height_clean = list(datos_height_clean)
datos_height_clean

['5\'3"',
 '5\'7"',
 '5\'8"',
 '5\'7"',
 '5\'7"',
 '5\'10"',
 '6\'2"',
 '5\'11"',
 '6\'0"',
 '6\'0"',
 '6\'0"',
 '5\'11"',
 '5\'7"',
 '6\'0"',
 '5\'9"',
 '6\'4"',
 '5\'7"',
 '5\'6"',
 '6\'3"',
 '6\'3"',
 '5\'10"',
 '5\'6"',
 '6\'0"',
 '6\'3"',
 '6\'3"',
 '5\'8"',
 '5\'9"',
 '6\'0"',
 '5\'8"',
 '5\'6"',
 '6\'0"',
 '6\'4"',
 '6\'0"',
 '5\'11"',
 '5\'9"',
 '6\'4"',
 '6\'0"',
 '6\'2"',
 '5\'8"',
 '5\'11"',
 '5\'11"',
 '6\'2"',
 '6\'0"',
 '5\'8"',
 '5\'8"',
 '5\'9"',
 '6\'2"',
 '5\'11"',
 '6\'5"',
 '6\'4"',
 '5\'6"',
 '5\'5"',
 '5\'7"',
 '5\'6"',
 '5\'11"',
 '6\'0"',
 '6\'1"',
 '5\'8"',
 '5\'8"',
 '5\'9"',
 '5\'6"',
 '5\'7"',
 '5\'10"',
 '5\'10"',
 '5\'10"',
 '6\'1"',
 '5\'9"',
 '6\'2"',
 '5\'10"',
 '6\'0"',
 '6\'1"',
 '5\'7"',
 '5\'9"',
 '5\'10"',
 '6\'0"',
 '5\'9"',
 '6\'3"',
 '5\'8"',
 '6\'5"',
 '5\'9"',
 '5\'9"',
 '5\'8"',
 '6\'2"',
 '6\'5"',
 '6\'2"',
 '6\'0"',
 '5\'10"',
 '6\'0"',
 '6\'1"',
 '5\'10"',
 '5\'6"',
 '5\'4"',
 '5\'10"',
 '5\'7"',
 '6\'4"',
 '6\'5"',
 '5\'8"',
 '6\'4"',
 '5

In [461]:
#creamos una función para convertir los datos
def str_a_float(dato):
    dato = dato.replace("'",".") # cambiamos el apostofe por punto
    dato = dato.replace("-",".") # camniamos el guion por punto
    dato = dato[:3] # accedemos solo a los primeros 3 elementos del str
    dato = float(dato) #convertimos el str a
    return dato

In [462]:
#aplicamos la función que creamos a la columna del data frame
df_altura_promedio["height_clean"] = df_altura_promedio["height_clean"].apply(lambda x: str_a_float(x))

ValueError: could not convert string to float: '5‚Ä'

In [27]:
# df_APE = df_Altura Promedio por Equipo
#df_APE = df.groupby('team')
#df_APE = df_APE['height_clean'].mean()
#df_APE